# 7 Working with Keras: A deep dive

In [20]:
import time
import pathlib
basedir = pathlib.Path("images")

import tensorflow as tf

## 7.3 Using built-in training and evaluation loops

**The standard workflow: `compile()`, `fit()`, `evaluate()`, `predict()`**

In [2]:
def get_mnist_model():
    inputs = tf.keras.Input(shape=(28 * 28,))
    features = tf.keras.layers.Dense(512, activation="relu")(inputs)
    features = tf.keras.layers.Dropout(0.5)(features)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
    model = tf.keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [3]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    train_images, train_labels, epochs=3, validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 [==============================] - 12s 7ms/step - loss: 0.2932 - accuracy: 0.9134 - val_loss: 0.1533 - val_accuracy: 0.9575
Epoch 2/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.1633 - accuracy: 0.9536 - val_loss: 0.1256 - val_accuracy: 0.9655
Epoch 3/3
313/313 [==============================] - 1s 3ms/step


### 7.3.1 Writing your own metrics

**Implementing a custom metric by subclassing the `Metric` class**

In [4]:
class RootMeanSquaredError(tf.keras.metrics.Metric):
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32"
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.0)
        self.total_samples.assign(0)

In [5]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", RootMeanSquaredError()],
)
model.fit(
    train_images, train_labels, epochs=3, validation_data=(val_images, val_labels)
)
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 [==============================] - 11s 6ms/step - loss: 0.2958 - accuracy: 0.9117 - rmse: 7.1837 - val_loss: 0.1441 - val_accuracy: 0.9594 - val_rmse: 7.3598
Epoch 2/3
1563/1563 [==============================] - 10s 7ms/step - loss: 0.1667 - accuracy: 0.9529 - rmse: 7.3545 - val_loss: 0.1209 - val_accuracy: 0.9666 - val_rmse: 7.3932
Epoch 3/3
313/313 [==============================] - 2s 6ms/step - loss: 0.1018 - accuracy: 0.9725 - rmse: 7.4331


---

## 7.4 Writing your own training and evaluation loops

### 7.4.1 Training versus inference

### 7.4.2 Low-level usage of metrics

In [6]:
metric = tf.keras.metrics.SparseCategoricalAccuracy()
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"result: {current_result:.2f}")

result: 1.00


In [7]:
values = [0, 1, 2, 3, 4]
mean_tracker = tf.keras.metrics.Mean()
for value in values:
    mean_tracker.update_state(value)
print(f"Mean of values: {mean_tracker.result():.2f}")

Mean of values: 2.00


### 7.4.3 A complete training and evaluation loop

**Writing a step-by-step training loop: the training step function**

In [8]:
model = get_mnist_model()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.RMSprop()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
loss_tracking_metric = tf.keras.metrics.Mean()

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

**Writing a step-by-step training loop: resetting the metrics**

In [9]:
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

**Writing a step-by-step training loop: the loop itself**

In [10]:
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.batch(32).enumerate()
epochs = 3                                  # ↑ adds an index like Python enumerate
for epoch in range(epochs):
    reset_metrics()
    #   ↓ an index allows us to keep count
    for i, (inputs_batch, targets_batch) in training_dataset:
        print(f"{i+1}/{len(train_images)//32}\r", end="")
        logs = train_step(inputs_batch, targets_batch)
    print(f"Results at the end of epoch {epoch}")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

Results at the end of epoch 0
...sparse_categorical_accuracy: 0.9146
...loss: 0.2917
Results at the end of epoch 1
...sparse_categorical_accuracy: 0.9541
...loss: 0.1652
Results at the end of epoch 2
...sparse_categorical_accuracy: 0.9623
...loss: 0.1428


**Writing a step-by-step evaluation loop**

In [13]:
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
times = []
for inputs_batch, targets_batch in val_dataset:
    t = time.time()
    logs = test_step(inputs_batch, targets_batch)
    times.append(time.time() - t)
print("Mean test_step time:", sum(times)/len(times))
print("Evaluation results:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

Mean test_step time: 0.013262062407911014
Evaluation results:
...val_sparse_categorical_accuracy: 0.9670
...val_loss: 0.1268


In [14]:
%%timeit
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)

4.58 s ± 306 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 7.4.4 Make it fast with tf.function

**Adding a `tf.function` decorator to our evaluation-step function**

In [15]:
@tf.function
def test_step_compiled(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()

    loss_tracking_metric.update_state(loss)
    logs["val_loss"] = loss_tracking_metric.result()
    return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
times = []
for inputs_batch, targets_batch in val_dataset:
    t = time.time()    
    logs = test_step_compiled(inputs_batch, targets_batch)
    times.append(time.time() - t)
print("Mean test_step time:", sum(times)/len(times)) # an order of magnitude smaller
print("Evaluation results:")
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

Mean test_step time: 0.004198075102541013
Evaluation results:
...val_sparse_categorical_accuracy: 0.9670
...val_loss: 0.1268


In [16]:
%%timeit
for inputs_batch, targets_batch in val_dataset:
    logs = test_step_compiled(inputs_batch, targets_batch)

1.01 s ± 33.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 7.4.5 Leveraging fit() with a custom training loop

**Implementing a custom training step to use with `fit()`**

In [17]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
loss_tracker = tf.keras.metrics.Mean(name="loss")

class CustomModel(tf.keras.Model):
    def train_step(self, data): # ← `train_step` is what we need to rewrit 
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = loss_fn(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))

        loss_tracker.update_state(loss)
        return {"loss": loss_tracker.result()}

    @property
    def metrics(self):
        return [loss_tracker]

In [18]:
inputs = tf.keras.Input(shape=(28 * 28,))
features = tf.keras.layers.Dense(512, activation="relu")(inputs)
features = tf.keras.layers.Dropout(0.5)(features)
outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.RMSprop())
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 9s 5ms/step - loss: 0.2935
Epoch 2/3
1563/1563 [==============================] - 7s 5ms/step - loss: 0.1628
Epoch 3/3
1563/1563 [==============================] - 7s 5ms/step - loss: 0.1407


In [28]:
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        inputs, targets = data
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            #           ↓ compiled loss
            loss = self.compiled_loss(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        #    ↓ compiled metrics for updating   
        self.compiled_metrics.update_state(targets, predictions)
        return {m.name: m.result() for m in self.metrics} # ← metrics for reporting

In [29]:
inputs = tf.keras.Input(shape=(28 * 28,))
features = tf.keras.layers.Dense(512, activation="relu")(inputs)
features = tf.keras.layers.Dropout(0.5)(features)
outputs = tf.keras.layers.Dense(10, activation="softmax")(features)
model = CustomModel(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
model.fit(train_images, train_labels, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.2939 - sparse_categorical_accuracy: 0.9125
Epoch 2/3
1563/1563 [==============================] - 9s 6ms/step - loss: 0.1650 - sparse_categorical_accuracy: 0.9543
Epoch 3/3
1563/1563 [==============================] - 10s 6ms/step - loss: 0.1384 - sparse_categorical_accuracy: 0.9637
